# Importations

In [1]:
# Test d'importation de SegmentationTask
try:
    from unetr.model_module import SegmentationTask
    print("SegmentationTask importé avec succès.")
except ImportError as e:
    print(f"Erreur d'importation : {e}")
except Exception as e:
    print(f"Erreur lors de l'importation de SegmentationTask: {e}")

In [2]:
from typing import List
import pydicom
from pydicom.dataset import Dataset
from io import BytesIO
import os 
import nibabel as nib
import numpy as np
from unetr.utilsUnetr.transforms import CropBedd, RandCropByPosNegLabeld, ResizeOrDoNothingd
from monai.transforms import Compose, Orientationd, ScaleIntensityRanged, CropForegroundd, ToTensord
from monai.transforms import RandFlipd, RandRotate90d, RandScaleIntensityd, RandShiftIntensityd
import torch
from monai.inferers import sliding_window_inference
from unetr.model_module import SegmentationTask
import monai.transforms as transforms
from rt_utils import RTStructBuilder

# Pipeline pour charger des dicoms en mémoire

In [4]:
def load_dicom_datasets(dicom_folder_path: str) -> List[Dataset]:
    """
    Charge tous les fichiers DICOM d'un dossier donné en datasets pydicom.
    """
    dicom_datasets = []
    for filename in os.listdir(dicom_folder_path):
        if filename.endswith('.dcm'):
            file_path = os.path.join(dicom_folder_path, filename)
            try:
                ds = pydicom.dcmread(file_path)
                dicom_datasets.append(ds)
            except Exception as e:
                print(f"Erreur lors de la lecture du fichier DICOM {filename}: {e}")
                continue  # ou lever une exception selon les besoins de votre application

    if not dicom_datasets:
        raise Exception("Aucun fichier DICOM valide trouvé dans le dossier spécifié.")

    return dicom_datasets

# Utilisation de la fonction
dicom_folder_path = '/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/RM/'
dicom_files = load_dicom_datasets(dicom_folder_path)
print(dicom_files[0])

# Fonctions

In [9]:
"""
def create_rtstruct(dicom_datasets: List[pydicom.dataset.Dataset], label):
    rtstruct = RTStructBuilder.create_new_from_memory(dicom_datasets)
    for i in range(1, int(np.max(label)) + 1):
        mask = np.where(label[0, :, :, :] == i, True, False) 
        rtstruct.add_roi(mask=mask, color=[255, 0, 0], name="GTV_MetIA_" + str(i))
    return rtstruct
"""
"""
def create_rtstruct(dicom_datasets: List[pydicom.dataset.Dataset], label):
    rtstruct = RTStructBuilder.create_new_from_memory(dicom_datasets)
    for i in range (1, int(np.max(label))+1):
        mask = np.where(label[0, :, :, :] == i, True, False) 
        rtstruct.add_roi(mask=np.rot90(np.where(mask==i, True, False)[0,:,:,:],3),color=[255, 0, 0], name="GTV_MetIA_"+str(i))
    return rtstruct
"""

def changeValueDicom(ds, un, deux, trois, quatre):
    listeValeurInterdite=[]
    valeur=0
    for i in range (0,len(ds[un ,deux].value)):
        valeur+=1
        try:
            print(ds[un, deux].value[i][un, quatre].value[:9] !="GTV_MetIA", ds[un, deux].value[i][un, quatre].value[:9])
            if ds[un, deux].value[i][un, quatre].value[:9] !="GTV_MetIA":
                listeValeurInterdite.append(ds[un, deux].value[i][un, trois].value)
            else:
                valeur, listeValeurInterdite=self.parcoursValue(valeur, listeValeurInterdite)
                ds[un, deux].value[i][un, trois].value=valeur
        except:
            valeur, listeValeurInterdite=self.parcoursValue(valeur, listeValeurInterdite)
            ds[un, deux].value[i][un, trois].value=valeur
        print(ds[un, deux].value[i])
    return ds
rtstruct_path = '/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META_MODIFIED/RS1.2.752.243.1.1.20230602094342956.1800.51115.dcm'
label = np.load('LabelARTuring.npy');
print(label.shape)

rtstruct = RTStructBuilder.create_from(dicom_series_path=dicom_folder_path, rt_struct_path=rtstruct_path)
for i in range (1, int(np.max(label))+1):
    rtstruct.add_roi(mask=np.rot90(np.where(label==i, True, False)[0,:,:,:],3),color=[255, 0, 0], name="GTV_MetIA_"+str(i))
rtstruct.save("./res.dcm")
ds = pydicom.dcmread(rtstruct_path)
#ds = self.changeValueDicom(ds, 0x3006, 0x0080, 0x0084, 0x0085)
ds = changeValueDicom(ds, 0x3006, 0x0020, 0x0022, 0x0026)
pydicom.filewriter.dcmwrite(rtstruct_path,ds,write_like_original=True)

# Suppression des petites metastases / faux positifs

In [3]:
dicom_folder_path = '/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/RM/'
rtstruct_path = '/Users/romain/Documents/P_R_O_J_E_C_T_S/IRM-Project/mbiaDataDownloads/DATA_VERITE_TERRAIN/META/RS1.2.752.243.1.1.20230602094342956.1800.51115.dcm'

In [62]:
label = np.load('labelAl.npy')
label.shape
import scipy.ndimage as ndimage
labeled_array, num_features = ndimage.label(label)
# afficher la feature 13
slices = ndimage.find_objects(labeled_array)
"""
for i in range(1, num_features + 1):
    print("voici la slice ",i-1,":")
    print(slices[i-1])
    roi_data = labeled_array[slices[i-1]]
    print(roi_data.shape)
    print(roi_data)
    print(np.unique(roi_data))
    print(i)
    print(np.sum(roi_data))
"""
#print(slices[24])
roi_data = label[0:1, 205:228, 209:235, 80:89]
#print(roi_data.shape)


# Calculer et afficher la taille de chaque feature
for i in range(num_features):
    if slices[i] is not None:
        # Calculer la taille de la région
        size = np.prod([(s.stop - s.start) for s in slices[i] if s is not None])
        print(f"Feature {i + 1}: Size = {size}")
        # Afficher des informations supplémentaires si nécessaire
        roi_data = labeled_array[slices[i]]
        #print(f"ROI data shape for feature {i + 1}: {roi_data.shape}")
        #print(f"Unique labels in ROI for feature {i + 1}: {np.unique(roi_data)}")
        #print(f"Sum of elements in ROI for feature {i + 1}: {np.sum(roi_data == i + 1)}")

print("--------------")

# Calculer et afficher la taille de chaque feature
for i in range(num_features):
    current_slice = slices[i]
    if current_slice is not None:
        current_region = labeled_array[current_slice]
        # Calculer la taille de la région
        size = np.sum(current_region == i + 1)
        print(f"Feature {i + 1}: Size = {size}")
        # Afficher des informations supplémentaires si nécessaire
        roi_data = labeled_array[slices[i]]


In [47]:
labeled_array, num_features = ndimage.label(label)
print("Number of features:", num_features)

# Trouver les slices pour chaque région
slices = ndimage.find_objects(labeled_array)

# Parcourir chaque feature pour évaluer et potentiellement supprimer les petites régions
for i in range(num_features):
    if slices[i] is not None:
        # Calculer la taille de la région
        size = np.prod([(s.stop - s.start) for s in slices[i] if s is not None])
        print(f"Feature {i + 1}: Size = {size}")
        
        # Vérifier si la taille est inférieure au seuil
        if size < 20:
            # Définir les valeurs de cette région à zéro
            labeled_array[slices[i]] = 0

In [49]:
labeled_array_res, num_features_res = ndimage.label(labeled_array)
print("Number of features after removing small regions:", num_features_res)

In [58]:
import matplotlib.pyplot as plt

plt.imshow(labeled_array[max(labeled_array.shape) // 2], cmap='gray')  # Afficher une coupe centrale si 3D
plt.title('Filtered Labels')
plt.colorbar()
plt.show()


In [6]:
rtstruct = RTStructBuilder.create_from(dicom_series_path=dicom_folder_path, rt_struct_path=rtstruct_path)

for i in range (1, np.max(label)+1):
    rtstruct.add_roi(mask=np.rot90(np.where(label[0]==i, True, False)[0,:,:,:],3),color=[255, 0, 0], name="GTV_MetIA_"+str(i))
rtstruct.save("./RESULTAT_RTSTRUCT_AL_SEUILLAGE.dcm")

ds = pydicom.dcmread("./RESULTAT_RTSTRUCT_AL_SEUILLAGE.dcm")
#ds = self.changeValueDicom(ds, 0x3006, 0x0080, 0x0084, 0x0085)
ds = changeValueDicom(ds, 0x3006, 0x0020, 0x0022, 0x0026)
pydicom.filewriter.dcmwrite("./RESULTAT_RTSTRUCT_AL_SEUILLAGE.dcm",ds,write_like_original=True)